In [3]:
import pandas as pd
import chromadb
import argparse
import pickle
from tqdm import tqdm
import os
import sys
import math


In [4]:
sys.path.append("../")

from workloads.inference import infer, create_id_to_abstract_dict
from utils.build_graph import build_graph


In [5]:
graph_path = "../data/graph.pickle"
print("Building graph...")


filtered_data_path = "../data/filtered_data.pickle"

with open(filtered_data_path, "rb") as f:
    filtered_data = pickle.load(f)

graph = build_graph(filtered_data)

with open(graph_path, "wb") as f:
    pickle.dump(graph, f)


Building graph...


In [7]:
id2abstract_dict = create_id_to_abstract_dict(filtered_data)


In [8]:
result_df = infer(
    "../data/chroma_dbs/",
    graph,
    "../workloads/workload.csv",
    "arxiv_vector",
    "abstracts",
    id2abstract_dict,
    10
)

100%|███████████████████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.25s/it]


In [12]:
result_df.head(10)

,paper_id,query,arxiv_vector,abstracts,hybrid
0,1802.07896,find papers written by < MarkN.Wegman > on < c...,"[2009.02383, 2302.04419, 2102.11271, 2106.0348...","[1802.07896, 1906.02896, 2102.11935, 2105.1467...","[2009.02383, 2302.04419, 2102.11271, 2106.0348..."
1,1802.07896,find papers from year 2019 on < cs.AI > about ...,"[2007.06007, 2304.09802, 1712.04473, 2011.0810...","[1802.07896, 1906.02896, 2102.11935, 2105.1467...","[2007.06007, 2304.09802, 1712.04473, 2011.0810..."
2,1901.03019,find papers written by < KenzoIshikawa > on < ...,"[1607.02058, 2104.1412, 1611.06118, 1607.01429...","[1901.03019, 1907.01264, 2301.0455, 1810.0328,...","[1607.02058, 2104.1412, 1611.06118, 1607.01429..."
3,1901.03019,find papers written by < ArisaKubota > from ye...,"[1510.02332, 1909.00123, 2210.04203, hep-ph/94...","[1901.03019, 1907.01264, 2301.0455, 1810.0328,...","[1510.02332, 1909.00123, 2210.04203, hep-ph/94..."
4,1911.08350,find papers titled < Solar Event Tracking with...,"[1911.0835, 2206.05561, astro-ph/0510377, 1812...","[1911.0835, 2210.04122, 2104.1395, 2211.02278,...","[1911.0835, 2206.05561, astro-ph/0510377, 1812..."
5,1911.08350,find papers written by < ToqiTahamidSarkerandJ...,"[2204.02828, 2302.12837, 2307.04285, 2303.0604...","[1911.0835, 2210.04122, 2104.1395, 2211.02278,...","[2204.02828, 2302.12837, 2307.04285, 2303.0604..."
6,2108.06554,find papers from year 2021 on < cs.CV > about ...,"[2009.12015, 1610.01924, 2003.13958, 1912.0719...","[2108.06554, 2203.02533, 1902.0867, 2006.13011...","[2009.12015, 1610.01924, 2003.13958, 1912.0719..."
7,2108.06554,find papers titled < Stacked Hourglass Network...,"[2108.06554, 2002.05773, 2002.05049, 2006.0793...","[2108.06554, 2203.02533, 1902.0867, 2006.13011...","[2108.06554, 2002.05773, 2002.05049, 2006.0793..."
8,1807.04394,find papers written by < MomonariKudo > from y...,"[2101.09632, hep-th/0107251, 2103.16309, 2111....","[1807.04394, 1804.09063, 1911.05653, 1902.0637...","[2101.09632, hep-th/0107251, 2103.16309, 2111...."
9,1807.04394,find papers about < pmod >,"[2302.12854, hep-ph/0301030, 1310.4904, hep-ph...","[1807.04394, 1804.09063, 1911.05653, 1902.0637...","[2302.12854, hep-ph/0301030, 1310.4904, hep-ph..."
